## Предсказания на привате

In [16]:
import pandas as pd
import numpy as np
import transformers
import torch

from sklearn import metrics
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
df = pd.read_csv('private_test.csv')
df = df.rename({'Фильм': 'movie', 'Сюжет': 'plot', 'Описание': 'description'}, axis=1)
df

,movie,description,plot
0,Пираты галактики Барракуда (2024),На Землю попадает пришелец из далёкой галактик...,На Землю попадает пришелец из далёкой галактик...
1,Брат (1997),Демобилизованный из армии Данила Багров возвра...,Осень 1996 года. Вернувшийся с Чеченской войны...
2,Смерч 2 (2024),Кейт Картер изучает метеорологию в Нью-Йорке. ...,Кейт Картер изучает метеорологию в Нью-Йорке. ...
3,Реквием по мечте (2000),"В драме ""Реквием по мечте"" Сара Голдфарб мечта...",Главные герои: мать — Сара (Эллен Бёрстин) и с...
4,Капитан Волконогов бежал (2021),В Ленинграде 1938 года капитан Фёдор Волконого...,"Сюрреалистический Ленинград, 1938 год. Главный..."
...,...,...,...
118,Великий Гэтсби (2013),Весной 1922 года Ник Каррауэй приезжает в Нью-...,"Ник Кэррауэй, выпускник Йельского университета..."
119,Пришельцы (2022),В этом захватывающем блокбастере о путешествия...,NaN
120,Загадочная история Бенджамина Баттона (2008),"Фильм рассказывает историю Бенджамина Баттона,...",В августе 2005 года Дейзи Фуллер находится на ...
121,Отряд самоубийц (2016),После смерти Супермена правительство решает да...,После смерти Супермена агент разведки Аманда У...


In [4]:
df.isnull().any()

movie          False
description    False
plot            True
dtype: bool

In [5]:
df['plot'].fillna(df['description'], inplace=True)
df.isnull().any()

C:\Users\Animados\AppData\Local\Temp\ipykernel_12216\2100915472.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['plot'].fillna(df['description'], inplace=True)


movie          False
description    False
plot           False
dtype: bool

In [6]:
import re

for i in range(len(df)):
    it_1 = re.sub(r'[\n]', ' ', df['plot'][i])
    it_2 = re.sub(r'\s+', ' ', it_1)
    df.loc[i, "plot"] = re.sub(r'\[[\d\w\.]+\]', '', it_2)

In [7]:
for i in range(len(df)):
    it_1 = re.sub(r'[\n]', ' ', df['description'][i])
    it_2 = re.sub(r'\s+', ' ', it_1)
    df.loc[i, "description"] = re.sub(r'\[[\d\w\.]+\]', '', it_2)

In [8]:
# объединим такие ячейки
df.loc[(df['description'].str.slice(0, 13) != df['plot'].str.slice(0, 13)), 'plot'] = df['description'] + ' ' + df['plot']

In [9]:
# посмотрим минимальное и максимальное количество токенов
plot_len_train = [len(str(i).split()) for i in df['plot']]
max(plot_len_train)

1291

In [10]:
min(plot_len_train)

28

In [11]:
MAX_LEN = 512

tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
class TestDataset(Dataset):

  def __init__(self, dataframe, tokenizer, max_len):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.text = self.data['plot']
    self.max_len = max_len

  def __len__(self):
    return len(self.text)

  def __getitem__(self, index):
    text = str(self.text[index])

    inputs = self.tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=self.max_len,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True
    )
    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]

    return {
        'ids': torch.tensor(ids, dtype=torch.long),
        'mask': torch.tensor(mask, dtype=torch.long),
        'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
    }

C:\Users\Animados\anaconda3\envs\tensorflow\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
public_set = TestDataset(df, tokenizer, MAX_LEN)
public_dataloader = DataLoader(public_set, batch_size=16, shuffle=False)

In [13]:
NUM_CLASSES = 14

class BERTClass(torch.nn.Module):
  def __init__(self, num_classes):
    super(BERTClass, self).__init__()
    # self.l1 = transformers.AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")
    # self.l1 = transformers.AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
    self.l1 = transformers.BertModel.from_pretrained('DeepPavlov/rubert-base-cased')
    self.l2 = torch.nn.Linear(768, num_classes)

  def forward(self, ids, mask, token_type_ids):
    x = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
    x = self.l2(x['pooler_output'])
    return x


model = BERTClass(NUM_CLASSES)
model.to(device)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [14]:
path_to_weights = './Task3_final24_Bert_25_split_LB_64.232_20epoch_seed_110_check.pth'
model.load_state_dict(torch.load(path_to_weights))

C:\Users\Animados\AppData\Local\Temp\ipykernel_12216\3870167170.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path_to_weights))


<All keys matched successfully>

In [17]:
model.eval()
fin_outputs = []

with torch.no_grad():
    for data in tqdm(public_dataloader):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)

        outputs = model(ids, mask, token_type_ids)

        fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

threshold = 0.85
fin_outputs = (np.array(fin_outputs) >= threshold).astype(int)

  0%|          | 0/8 [00:00<?, ?it/s]C:\Users\Animados\anaconda3\envs\tensorflow\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 8/8 [00:03<00:00,  2.34it/s]


In [18]:
genres = ['боевик', 'детектив', 'драма', 'история', 'комедия', 'криминал',
       'мелодрама', 'мультфильм', 'мюзикл', 'приключения', 'триллер',
       'ужасы', 'фантастика', 'фэнтези']

In [19]:
def replace_and_join(array, genres):
    result = []
    for row in array:
        genre_indices = np.where(row == 1)[0]
        genre_names = [genres[i] for i in genre_indices]
        result.append(", ".join(genre_names))
    return result

answers = replace_and_join(fin_outputs, genres)

In [20]:
answers_df = df[['movie']].copy()
answers_df['genres'] = answers
answers_df = answers_df.rename({'movie': 'Фильм', 'genres': 'Жанры'}, axis=1)
answers_df

,Фильм,Жанры
0,Пираты галактики Барракуда (2024),"мультфильм, приключения, фантастика"
1,Брат (1997),"боевик, комедия, криминал"
2,Смерч 2 (2024),"драма, приключения, фантастика"
3,Реквием по мечте (2000),"драма, комедия, криминал"
4,Капитан Волконогов бежал (2021),"боевик, детектив, драма, криминал"
...,...,...
118,Великий Гэтсби (2013),"драма, мелодрама, фэнтези"
119,Пришельцы (2022),"боевик, приключения, фантастика"
120,Загадочная история Бенджамина Баттона (2008),"драма, история, мелодрама"
121,Отряд самоубийц (2016),"боевик, комедия, приключения, фантастика"


In [21]:
seed = 110
answers_df.to_csv(f'submit_private_seed_{seed}_threshold85.csv', index=False)